# Mean bias correction

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [2]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = '00:30:00'
cores = 24
memory = str(4 * cores) + 'GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=['-q normal',
                                           '-P w42',
                                           '-l ncpus='+str(cores),
                                           '-l mem='+str(memory),
                                           '-l storage=gdata/w42+gdata/rt52+gdata/fs38'],
                     local_directory='$TMPDIR',
                     job_directives_skip=["select"])
                     # python=os.environ["DASK_PYTHON"])

/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36987 instead
  warnings.warn(


In [3]:
cluster.scale(jobs=1)
client = Client(cluster)

In [4]:
client#.restart()

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/36987/status,
Dashboard: /proxy/36987/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.2:41381,Workers: 0
Dashboard: /proxy/36987/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [1]:
import xarray as xr

In [2]:
path_model = "/g/data/w42/dr6273/work/data/CMIP6/ACCESS-ESM1-5/historical/day/"
path_era = "/g/data/w42/dr6273/work/data/era5/regridded/CMIP6/"

In [10]:
def open_model(var):
    """
    Open ACCESS zarr collection
    """
    return xr.open_zarr(
        path_model + var + "/" + var + "_day_ACCESS-ESM1-5_historical_1950-2014_NEM.zarr",
        consolidated=True
    )

In [4]:
def open_era(var):
    """
    Open ERA5 data regridded to ACCESS grid
    """
    return xr.open_mfdataset(
        path_era + var + "_era5_ACCESS-ESM1-5_1959-2014_NEM.nc"
    )

### Temperature (2m)

In [8]:
tas_model = open_model("tas")

In [9]:
tas_model

<xarray.Dataset>
Dimensions:  (lat: 29, lon: 12, member: 40, time: 23741)
Coordinates:
  * lat      (lat) float64 -45.0 -43.75 -42.5 -41.25 ... -12.5 -11.25 -10.0
  * lon      (lon) float64 133.1 135.0 136.9 138.8 ... 148.1 150.0 151.9 153.8
  * member   (member) int64 1 2 3 4 5 6 7 8 9 10 ... 32 33 34 35 36 37 38 39 40
  * time     (time) datetime64[ns] 1950-01-01T12:00:00 ... 2014-12-31T12:00:00
Data variables:
    tas      (member, time, lat, lon) float32 dask.array<chunksize=(5, 18262, 29, 12), meta=np.ndarray>

In [ ]:
model_tas.to_netcdf(
    "/g/data/w42/dr6273/work/data/era5/regridded/CMIP6/2t_era5_ACCESS-ESM1-5_1959-2014_NEM.nc"
)